<a href="https://colab.research.google.com/github/sambhavigit/GCP_vision/blob/main/LKMAIGCP/Batch1/text_to_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vertex AI Model Garden - Text To Video

<table align="left">
  <td>
Colab Notebook

## Overview

This notebook demonstrates deploying the pre-trained [Text To Video](https://huggingface.co/docs/diffusers/main/en/api/pipelines/text_to_video) model on Vertex AI for online prediction.

### Objective

- Upload the model to [Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction).
- Deploy the model on [Endpoint](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints).
- Run online predictions for text-to-video.


In [2]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform

    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

    from google.colab import auth as google_auth

    google_auth.authenticate_user()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.0/321.0 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.8 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.1
    Uninstalling shapely-2.0.1:
      Successfully uninstalled shapely-2.0.1


In [1]:
# Cloud project id.
PROJECT_ID = "acn-lkmaigcp"  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = "us-central1"  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output. Fill it without the 'gs://' prefix.
GCS_BUCKET = ""  # @param {type:"string"}

Initialize Vertex AI API:

In [2]:
from google.cloud import aiplatform

aiplatform.init(project="acn-lkmaigcp", location="us-central1", staging_bucket=GCS_BUCKET)

### Define constants

In [3]:
# The pre-built serving docker image. It contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-diffusers-serve"

In [4]:
from google.cloud import aiplatform


def deploy_model(model_id, task):
    model_name = "text-to-video"
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {
        "MODEL_ID": model_id,
        "TASK": task,
    }
    # If the model_id is a GCS path, use artifact_uri to pass it to serving docker.
    artifact_uri = model_id if model_id.startswith("gs://") else None
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/diffusers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
        artifact_uri=artifact_uri,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type="n1-standard-8",
        accelerator_type="NVIDIA_TESLA_T4",
        accelerator_count=1,
        deploy_request_timeout=1800,
    )
    return model, endpoint

In [5]:
model, endpoint = deploy_model(
    model_id="damo-vilab/text-to-video-ms-1.7b", task="text-to-video"
)

Creating Endpoint


INFO:google.cloud.aiplatform.models:Creating Endpoint


Create Endpoint backing LRO: projects/874849068168/locations/us-central1/endpoints/63580359387774976/operations/2050065474670034944


INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/874849068168/locations/us-central1/endpoints/63580359387774976/operations/2050065474670034944


Endpoint created. Resource name: projects/874849068168/locations/us-central1/endpoints/63580359387774976


...

INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/874849068168/locations/us-central1/endpoints/63580359387774976


Deploy Endpoint model backing LRO: projects/874849068168/locations/us-central1/endpoints/63580359387774976/operations/5971574850202894336


INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/874849068168/locations/us-central1/endpoints/63580359387774976/operations/5971574850202894336


Endpoint model deployed. Resource name: projects/874849068168/locations/us-central1/endpoints/63580359387774976


INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/874849068168/locations/us-central1/endpoints/63580359387774976


In [1]:
instances = [
    {"prompt": "Spiderman and batman is surfing"},
]
response = endpoint.predict(instances=instances)

NameError: ignored

In [14]:
from IPython.display import HTML

html = ""
for video in response.predictions:
    html += "<video controls>"
    html += f'<source src="data:video/mp4;base64,{video}" type="video/mp4">'
    html += "</video>"
HTML(html)

### Clean up resources if not using this model further for any video creation

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()